In [1]:
!git clone -b mpii https://github.com/Cancogang69/clone_hrnet.git

Cloning into 'clone_hrnet'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 224 (delta 113), reused 211 (delta 102), pack-reused 0
Receiving objects: 100% (224/224), 7.38 MiB | 27.88 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [2]:
%pip install --upgrade pip
%cd ./clone_hrnet
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.
/kaggle/working/clone_hrnet
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 9.1 MB/s eta 0:00:00:00:01
  Created wheel for EasyDict: filename=easydict-1.7-py3-none-any.whl size=6105 sha256=2544767050451e01c184521e29e62d4bff9e35a8940c5bf1fc703681343d95d3
  Stored in directory: /root/.cache/pip/wheels/ce/67/6e/cbc2379c9b1cd9d7d7068eba7d8f5d5befcc82e5f691883c15
Successfully built EasyDict
  Attempting uninstall: EasyDict
    Found existing installation: easydict 1.13
    Uninstalling easydict-1.13:
      Successfully uninstalled easydict-1.13
Note: you may need to restart the kernel to use updated packages.


In [3]:
!mkdir output 
!mkdir log
!ls

LICENSE    _config.yml	experiments  lib  output	    tools
README.md  demo		figures      log  requirements.txt  visualization


# Import library

In [4]:
import sys
import argparse
import os
import pprint
import shutil
from yacs.config import CfgNode as CN

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

from lib.config import update_config
from lib.core.loss import JointsMSELoss
from lib.core.function import train
from lib.core.function import validate
from lib.utils.utils import get_optimizer
from lib.utils.utils import save_checkpoint
from lib.utils.utils import create_logger
from lib.utils.utils import get_model_summary

import lib.dataset
from lib.models.pose_hrnet import get_pose_net

/kaggle/working/clone_hrnet/lib/models/pose_hrnet.py:487: SyntaxWarning: "is" with a literal. Did you mean "=="?
  or self.pretrained_layers[0] is '*':


In [5]:
!ls

LICENSE    _config.yml	experiments  lib  output	    tools
README.md  demo		figures      log  requirements.txt  visualization


# Train Configs

In [8]:
cfg = CN()

cfg.OUTPUT_DIR = 'output'
cfg.LOG_DIR = 'log'
cfg.DATA_DIR = ''
cfg.GPUS = (0,)
cfg.WORKERS = 24
cfg.PRINT_FREQ = 100
cfg.AUTO_RESUME = True
cfg.PIN_MEMORY = True
cfg.RANK = 0

# Cudnn related params
cfg.CUDNN = CN()
cfg.CUDNN.BENCHMARK = True
cfg.CUDNN.DETERMINISTIC = False
cfg.CUDNN.ENABLED = True

# common params for NETWORK
cfg.MODEL = CN()
cfg.MODEL.NAME = 'pose_hrnet'
cfg.MODEL.INIT_WEIGHTS = True
cfg.MODEL.PRETRAINED = '/kaggle/input/hrnet-w32-pretrained/hrnet_w32-36af842e.pth'
cfg.MODEL.NUM_JOINTS = 16
cfg.MODEL.TAG_PER_JOINT = True
cfg.MODEL.TARGET_TYPE = 'gaussian'
cfg.MODEL.IMAGE_SIZE = [256, 256]  # width * height, ex: 192 * 256
cfg.MODEL.HEATMAP_SIZE = [64, 64]  # width * height, ex: 24 * 32
cfg.MODEL.SIGMA = 2

cfg.MODEL.EXTRA = CN()
cfg.MODEL.EXTRA.PRETRAINED_LAYERS = [
  'conv1',
  'bn1',
  'conv2',
  'bn2',
  'layer1',
  'transition1',
  'stage2',
  'transition2',
  'stage3',
  'transition3',
  'stage4'
]
cfg.MODEL.EXTRA.STEM_INPLANES = 64
cfg.MODEL.EXTRA.FINAL_CONV_KERNEL = 1

cfg.MODEL.EXTRA.STAGE2 = CN()
cfg.MODEL.EXTRA.STAGE2.NUM_MODULES = 1
cfg.MODEL.EXTRA.STAGE2.NUM_BRANCHES = 2
cfg.MODEL.EXTRA.STAGE2.NUM_BLOCKS = [4, 4]
cfg.MODEL.EXTRA.STAGE2.NUM_CHANNELS = [32, 64]
cfg.MODEL.EXTRA.STAGE2.BLOCK = 'BASIC'
cfg.MODEL.EXTRA.STAGE2.FUSE_METHOD = 'SUM'

cfg.MODEL.EXTRA.STAGE3 = CN()
cfg.MODEL.EXTRA.STAGE3.NUM_MODULES = 4
cfg.MODEL.EXTRA.STAGE3.NUM_BRANCHES = 3
cfg.MODEL.EXTRA.STAGE3.NUM_BLOCKS = [4, 4, 4]
cfg.MODEL.EXTRA.STAGE3.NUM_CHANNELS = [32, 64, 128]
cfg.MODEL.EXTRA.STAGE3.BLOCK = 'BASIC'
cfg.MODEL.EXTRA.STAGE3.FUSE_METHOD = 'SUM'

cfg.MODEL.EXTRA.STAGE4 = CN()
cfg.MODEL.EXTRA.STAGE4.NUM_MODULES = 3
cfg.MODEL.EXTRA.STAGE4.NUM_BRANCHES = 4
cfg.MODEL.EXTRA.STAGE4.NUM_BLOCKS = [4, 4, 4, 4]
cfg.MODEL.EXTRA.STAGE4.NUM_CHANNELS = [32, 64, 128, 256]
cfg.MODEL.EXTRA.STAGE4.BLOCK = 'BASIC'
cfg.MODEL.EXTRA.STAGE4.FUSE_METHOD = 'SUM'

cfg.LOSS = CN()
cfg.LOSS.USE_OHKM = False
cfg.LOSS.TOPK = 8
cfg.LOSS.USE_TARGET_WEIGHT = True
cfg.LOSS.USE_DIFFERENT_JOINTS_WEIGHT = False

# DATASET related params
cfg.DATASET = CN()
cfg.DATASET.ROOT = '/kaggle/input/mpii-2014'
cfg.DATASET.DATASET = 'mpii'
cfg.DATASET.TRAIN_SET = 'train'
cfg.DATASET.TEST_SET = 'valid'
cfg.DATASET.DATA_FORMAT = 'jpg'
cfg.DATASET.HYBRID_JOINTS_TYPE = ''
cfg.DATASET.SELECT_DATA = False

# training data augmentation
cfg.DATASET.COLOR_RGB = True
cfg.DATASET.FLIP = True
cfg.DATASET.NUM_JOINTS_HALF_BODY = 8
cfg.DATASET.PROB_HALF_BODY = -1.0
cfg.DATASET.ROT_FACTOR = 30
cfg.DATASET.SCALE_FACTOR = 0.25

# train
cfg.TRAIN = CN()

cfg.TRAIN.LR_FACTOR = 0.1
cfg.TRAIN.LR_STEP = [170, 200]
cfg.TRAIN.LR = 0.001

cfg.TRAIN.OPTIMIZER = 'adam'
cfg.TRAIN.MOMENTUM = 0.9
cfg.TRAIN.WD = 0.0001
cfg.TRAIN.NESTEROV = False
cfg.TRAIN.GAMMA1 = 0.99
cfg.TRAIN.GAMMA2 = 0.0

cfg.TRAIN.BEGIN_EPOCH = 0
cfg.TRAIN.END_EPOCH = 100

cfg.TRAIN.RESUME = False
cfg.TRAIN.CHECKPOINT = ''

cfg.TRAIN.BATCH_SIZE_PER_GPU = 32
cfg.TRAIN.SHUFFLE = True

# testing
cfg.TEST = CN()

cfg.TEST.BATCH_SIZE_PER_GPU = 32
cfg.TEST.FLIP_TEST = True
cfg.TEST.POST_PROCESS = True
cfg.TEST.SHIFT_HEATMAP = True

cfg.TEST.USE_GT_BBOX = False

# nms
cfg.TEST.IMAGE_THRE = 0.1
cfg.TEST.NMS_THRE = 0.6
cfg.TEST.SOFT_NMS = False
cfg.TEST.OKS_THRE = 0.5
cfg.TEST.IN_VIS_THRE = 0.0
cfg.TEST.COCO_BBOX_FILE = ''
cfg.TEST.BBOX_THRE = 1.0
cfg.TEST.MODEL_FILE = ''

# debug
cfg.DEBUG = CN()
cfg.DEBUG.DEBUG = False
cfg.DEBUG.SAVE_BATCH_IMAGES_GT = False
cfg.DEBUG.SAVE_BATCH_IMAGES_PRED = False
cfg.DEBUG.SAVE_HEATMAPS_GT = False
cfg.DEBUG.SAVE_HEATMAPS_PRED = False

# model name
model_name = "w32_256x256_adam_lr1e-3"

# Train

In [ ]:
logger, final_output_dir, tb_log_dir = create_logger(
    cfg, model_name, 'valid')

logger.info(cfg)

# cudnn related setting
cudnn.benchmark = cfg.CUDNN.BENCHMARK
torch.backends.cudnn.deterministic = cfg.CUDNN.DETERMINISTIC
torch.backends.cudnn.enabled = cfg.CUDNN.ENABLED

model = get_pose_net(cfg, is_train=True)

writer_dict = {
    'writer': SummaryWriter(log_dir=tb_log_dir),
    'train_global_steps': 0,
    'valid_global_steps': 0,
}

dump_input = torch.rand(
    (1, 3, cfg.MODEL.IMAGE_SIZE[1], cfg.MODEL.IMAGE_SIZE[0])
)
writer_dict['writer'].add_graph(model, (dump_input, ))

logger.info(get_model_summary(model, dump_input))

model = torch.nn.DataParallel(model, device_ids=cfg.GPUS).cuda()

# define loss function (criterion) and optimizer
criterion = JointsMSELoss(
    use_target_weight=cfg.LOSS.USE_TARGET_WEIGHT
).cuda()

# Data loading code
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)
train_dataset = lib.dataset.mpii(
    cfg, cfg.DATASET.ROOT, cfg.DATASET.TRAIN_SET, True,
    transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
)
valid_dataset = lib.dataset.mpii(
    cfg, cfg.DATASET.ROOT, cfg.DATASET.TEST_SET, False,
    transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=cfg.TRAIN.BATCH_SIZE_PER_GPU*len(cfg.GPUS),
    shuffle=cfg.TRAIN.SHUFFLE,
    num_workers=cfg.WORKERS,
    pin_memory=cfg.PIN_MEMORY
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=cfg.TEST.BATCH_SIZE_PER_GPU*len(cfg.GPUS),
    shuffle=False,
    num_workers=cfg.WORKERS,
    pin_memory=cfg.PIN_MEMORY
)

best_perf = 0.0
best_model = False
last_epoch = -1
optimizer = get_optimizer(cfg, model)
begin_epoch = cfg.TRAIN.BEGIN_EPOCH
checkpoint_file = os.path.join(
    final_output_dir, 'checkpoint.pth'
)

if cfg.AUTO_RESUME and os.path.exists(checkpoint_file):
    logger.info("=> loading checkpoint '{}'".format(checkpoint_file))
    checkpoint = torch.load(checkpoint_file)
    begin_epoch = checkpoint['epoch']
    best_perf = checkpoint['perf']
    last_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])

    optimizer.load_state_dict(checkpoint['optimizer'])
    logger.info("=> loaded checkpoint '{}' (epoch {})".format(
        checkpoint_file, checkpoint['epoch']))

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, cfg.TRAIN.LR_STEP, cfg.TRAIN.LR_FACTOR,
    last_epoch=last_epoch
)

for epoch in range(begin_epoch, cfg.TRAIN.END_EPOCH):
    lr_scheduler.step()

    # train for one epoch
    train(cfg, train_loader, model, criterion, optimizer, epoch,
            final_output_dir, tb_log_dir, writer_dict)


    # evaluate on validation set
    perf_indicator = validate(
        cfg, valid_loader, valid_dataset, model, criterion,
        final_output_dir, tb_log_dir, writer_dict
    )

    if perf_indicator >= best_perf:
        best_perf = perf_indicator
        best_model = True
    else:
        best_model = False

    logger.info('=> saving checkpoint to {}'.format(final_output_dir))
    save_checkpoint({
        'epoch': epoch + 1,
        'model': cfg.MODEL.NAME,
        'state_dict': model.state_dict(),
        'best_state_dict': model.module.state_dict(),
        'perf': perf_indicator,
        'optimizer': optimizer.state_dict(),
    }, best_model, final_output_dir)

final_model_state_file = os.path.join(
    final_output_dir, 'final_state.pth'
)
logger.info('=> saving final model state to {}'.format(
    final_model_state_file)
)
torch.save(model.module.state_dict(), final_model_state_file)
writer_dict['writer'].close()

AUTO_RESUME: True
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  COLOR_RGB: True
  DATASET: mpii
  DATA_FORMAT: jpg
  FLIP: True
  HYBRID_JOINTS_TYPE: 
  NUM_JOINTS_HALF_BODY: 8
  PROB_HALF_BODY: -1.0
  ROOT: /kaggle/input/mpii-2014
  ROT_FACTOR: 30
  SCALE_FACTOR: 0.25
  SELECT_DATA: False
  TEST_SET: valid
  TRAIN_SET: train
DATA_DIR: 
DEBUG:
  DEBUG: False
  SAVE_BATCH_IMAGES_GT: False
  SAVE_BATCH_IMAGES_PRED: False
  SAVE_HEATMAPS_GT: False
  SAVE_HEATMAPS_PRED: False
GPUS: (0,)
LOG_DIR: log
LOSS:
  TOPK: 8
  USE_DIFFERENT_JOINTS_WEIGHT: False
  USE_OHKM: False
  USE_TARGET_WEIGHT: True
MODEL:
  EXTRA:
    FINAL_CONV_KERNEL: 1
    PRETRAINED_LAYERS: ['conv1', 'bn1', 'conv2', 'bn2', 'layer1', 'transition1', 'stage2', 'transition2', 'stage3', 'transition3', 'stage4']
    STAGE2:
      BLOCK: BASIC
      FUSE_METHOD: SUM
      NUM_BLOCKS: [4, 4]
      NUM_BRANCHES: 2
      NUM_CHANNELS: [32, 64]
      NUM_MODULES: 1
    STAGE3:
      BLOCK: BASIC
      FUS

=> creating output/mpii/pose_hrnet/w32_256x256_adam_lr1e-3
=> creating log/mpii/pose_hrnet/w32_256x256_adam_lr1e-3_2024-05-07-16-20


=> init weights from normal distribution
=> init weights from normal distribution
=> loading pretrained model /kaggle/input/hrnet-w32-pretrained/hrnet_w32-36af842e.pth
=> loading pretrained model /kaggle/input/hrnet-w32-pretrained/hrnet_w32-36af842e.pth

Total Parameters: 28,536,080
----------------------------------------------------------------------------------------------------------------------------------
Total Multiply Adds (For Convolution and Linear Layers only): 9.4931640625 GFLOPs
----------------------------------------------------------------------------------------------------------------------------------
Number of Layers
Conv2d : 293 layers   BatchNorm2d : 292 layers   ReLU : 261 layers   Bottleneck : 4 layers   BasicBlock : 104 layers   Upsample : 28 layers   HighResolutionModule : 8 layers   

Total Parameters: 28,536,080
----------------------------------------------------------------------------------------------------------------------------------
Total Multiply Ad

!zip -r file.zip directory_path